In [1]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
from requests.exceptions import RequestException

In [2]:
coingecko_url = 'https://api.coingecko.com/api/v3'

In [9]:
import os
import requests
import joblib
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Get API key from environment
API_KEY = os.getenv("COINGECKO_API_KEY")

if not API_KEY:
    raise ValueError("🚨 Missing API key! Make sure COINGECKO_API_KEY is set in your .env file.")

# API endpoint
url = "https://pro-api.coingecko.com/api/v3/coins/top_gainers_losers"

headers = {
    "x-cg-pro-api-key": API_KEY,
    "accept": "application/json"
}

params = {"vs_currency": "usd"}

# Fetch data
response = requests.get(url, headers=headers, params=params)

if response.status_code != 200:
    print("❌ Status code:", response.status_code)
    print("Response:", response.text)
    response.raise_for_status()

data = response.json()

# Save data with timestamp
SAVE_FILE = r"C:\database\top_gainers_losers.joblib"

payload = {
    "timestamp": datetime.utcnow().isoformat(),
    "data": data
}

joblib.dump(payload, SAVE_FILE)
print(f"✅ Data saved to {SAVE_FILE} at {payload['timestamp']}")

# Optional: preview top gainers
#if "top_gainers" in data:
 ###     print(f"{coin['name']} ({coin['symbol'].upper()}) — ${coin['usd']:.2f} ({coin['usd_24h_change']:.2f}%)")


C:\Users\daddy brian\AppData\Local\Temp\ipykernel_10772\1277355482.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\database\\top_gainers_losers.joblib'

In [7]:
import joblib

# Path to saved file
SAVE_FILE = r"C:\database\top_gainers_losers.joblib"

# Load saved payload
payload = joblib.load(SAVE_FILE)
data = payload["data"]

print(f"📦 Data last updated at: {payload['timestamp']}\n")

# Extract gainers & losers
top_gainers = data.get("top_gainers", [])
top_losers = data.get("top_losers", [])

# Display gainers
print("📈 Top 5 Gainers:")
print("-" * 40)
for coin in top_gainers[:5]:
    name = coin.get("name", "Unknown")
    symbol = coin.get("symbol", "").upper()
    price = coin.get("usd", 0)
    change = coin.get("usd_24h_change", 0)
    rank = coin.get("market_cap_rank", "N/A")
    print(f"{name} ({symbol}) — ${price:,.2f} ({change:+.2f}%) [Rank {rank}]")

# Display losers
print("\n📉 Top 5 Losers:")
print("-" * 40)
for coin in top_losers[:5]:
    name = coin.get("name", "Unknown")
    symbol = coin.get("symbol", "").upper()
    price = coin.get("usd", 0)
    change = coin.get("usd_24h_change", 0)
    rank = coin.get("market_cap_rank", "N/A")
    print(f"{name} ({symbol}) — ${price:,.2f} ({change:+.2f}%) [Rank {rank}]")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\database\\top_gainers_losers.joblib'

In [12]:
import joblib
import os

# Define the full path to your joblib file
joblib_path = "database/top_gainers_losers.joblib"

# Check if file exists before loading
if not os.path.exists(joblib_path):
    print(f"⚠️ File not found at: {joblib_path}")
else:
    try:
        # Load joblib file
        data = joblib.load(joblib_path)

        # Print metadata
        print("📅 Last updated:", data.get("timestamp", "Unknown"))
        print("📰 Total articles:", len(data.get("articles", [])))

        # Example: show the first headline
        if data.get("articles"):
            first_article = data["articles"][0]
            print("\n🔥 First Headline:", first_article.get("title", "No title"))
            print("🔗 Link:", first_article.get("link", "No link"))
            print("📈 Sentiment:", first_article.get("sentiment", "N/A"))
        else:
            print("No articles found in the file.")

    except Exception as e:
        print(f"❌ Error loading joblib file: {e}")



📅 Last updated: 2025-10-26T22:54:04.698487
📰 Total articles: 0
No articles found in the file.


In [17]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
API_KEY = os.getenv("COINGECKO_API_KEY")

if not API_KEY:
    raise ValueError("🚨 Missing API key! Make sure COINGECKO_API_KEY is set in your .env file.")

# API endpoint for top 100 coins
url = "https://pro-api.coingecko.com/api/v3/coins/markets"

headers = {
    "x-cg-pro-api-key": API_KEY,
    "accept": "application/json"
}

params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 100,
    "page": 1,
    "sparkline": "false"
}

# Fetch data
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()

data = response.json()

# Display only coin name and current price
print("🪙 Top 100 Crypto Prices (USD):\n")
for coin in data:
    name = coin.get("name", "Unknown")
    price = coin.get("current_price", "N/A")
    print(f"{name}: ${price:,.2f}")


🪙 Top 100 Crypto Prices (USD):

Bitcoin: $111,440.00
Ethereum: $3,930.72
Tether: $1.00
XRP: $2.60
BNB: $1,107.71
Solana: $192.10
USDC: $1.00
Lido Staked Ether: $3,930.60
Dogecoin: $0.20
TRON: $0.30
Cardano: $0.65
Wrapped stETH: $4,786.85
Wrapped Bitcoin: $111,488.00
Wrapped Beacon ETH: $4,246.42
Chainlink: $18.01
Hyperliquid: $40.83
Wrapped eETH: $4,245.85
Stellar: $0.33
Ethena USDe: $1.00
Bitcoin Cash: $507.67
USDS: $1.00
Sui: $2.51
Binance Bridged USDT (BNB Smart Chain): $1.00
WETH: $3,933.54
Avalanche: $19.52
LEO Token: $9.00
Coinbase Wrapped BTC: $111,520.00
Litecoin: $96.16
Hedera: $0.17
USDT0: $1.00
Figure Heloc: $0.50
WhiteBIT Coin: $42.36
Monero: $328.45
Shiba Inu: $0.00
Cronos: $0.15
Toncoin: $2.16
Mantle: $1.64
Ethena Staked USDe: $1.20
Dai: $1.00
Polkadot: $3.07
Zcash: $272.67
MemeCore: $2.30
Bittensor: $387.42
Uniswap: $6.18
World Liberty Financial: $0.13
OKB: $168.88
Aave: $227.60
sUSDS: $1.07
Ethena: $0.47
Bitget Token: $4.67
Pepe: $0.00
USD1: $1.00
NEAR Protocol: $2.28
B

In [4]:
import os
import time
import joblib
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# -------------------------------
# 🧠 Simple Crypto Sentiment Classifier
# -------------------------------
def classify_sentiment(headline: str) -> str:
    """Classify crypto news headline as bullish, bearish, or neutral."""
    bullish_keywords = [
        "surge", "rally", "soar", "gain", "bull", "increase", "rise", "positive",
        "record", "high", "jump", "growth", "breakout", "buy", "invest", "pump"
    ]
    bearish_keywords = [
        "drop", "fall", "crash", "bear", "decline", "loss", "down", "negative",
        "sell", "dump", "fear", "panic", "collapse", "recession", "dip"
    ]

    text = headline.lower()
    bull_score = sum(1 for word in bullish_keywords if word in text)
    bear_score = sum(1 for word in bearish_keywords if word in text)

    if bull_score > bear_score:
        return "🟢 Bullish"
    elif bear_score > bull_score:
        return "🔴 Bearish"
    else:
        return "⚪ Neutral"


# -------------------------------
# 🌐 Fetch and Save CoinDesk Headlines
# -------------------------------
def fetch_and_save_coindesk_news(limit: int = 50, save_file: str = "coindesk_news.joblib"):
    """Fetch latest CoinDesk news headlines, classify sentiment, and save to joblib."""

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    url = "https://www.coindesk.com/"

    print("\n🌍 Fetching CoinDesk headlines...\n")

    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "h3")))

        soup = BeautifulSoup(driver.page_source, "html.parser")
        headlines = soup.find_all("h3")

        print(f"✅ Found {len(headlines)} headlines on CoinDesk.\n")

        articles = []
        for i, headline in enumerate(headlines[:limit], 1):
            try:
                title = headline.text.strip()
                parent_link = headline.find_parent("a")
                href = parent_link["href"] if parent_link and parent_link.get("href") else None
                full_link = url + href if href and href.startswith("/") else href

                sentiment = classify_sentiment(title)

                articles.append({
                    "title": title,
                    "link": full_link or "No link found",
                    "sentiment": sentiment
                })

                print(f"{i}. {title}")
                print(f"   → {full_link}")
                print(f"   Sentiment: {sentiment}\n")

            except Exception as e:
                print(f"⚠️ Error processing headline {i}: {str(e)}")

        # Save to joblib with timestamp
        data = {
            "timestamp": datetime.utcnow().isoformat(),
            "source": "CoinDesk",
            "articles": articles
        }

        joblib.dump(data, save_file)
        print(f"\n💾 Saved {len(articles)} headlines to '{save_file}' at {data['timestamp']}")

    except Exception as e:
        print(f"❌ Error fetching CoinDesk news: {str(e)}")

    finally:
        driver.quit()


# -------------------------------
# 🔁 Auto-run every hour (optional)
# -------------------------------
#if __name__ == "__main__":
#    while True:
 #       fetch_and_save_coindesk_news(limit=50)
  #      print("\n⏰ Waiting 1 hour before next fetch...\n")
   #     time.sleep(3600)  # Run every hour


In [6]:
import joblib
import os

JOBLIB_FILE = "top_gainers_losers.joblib"

if not os.path.exists(JOBLIB_FILE):
    print(f"⚠️ File '{JOBLIB_FILE}' not found. Please run the scraper first.")
else:
    try:
        # Load joblib data
        data = joblib.load(JOBLIB_FILE)

        # Display metadata
        print("📅 Last Updated:", data.get("timestamp", "Unknown"))
        print("📰 Source:", data.get("source", "Unknown"))
        print(f"🧾 Total Articles: {len(data.get('articles', []))}\n")

        # Display all (or some) headlines
        articles = data.get("articles", [])
        if not articles:
            print("No articles found in joblib file.")
        else:
            for i, article in enumerate(articles[:50], 1):  # show up to 50 headlines
                title = article.get("title", "No title")
                link = article.get("link", "No link")
                sentiment = article.get("sentiment", "⚪ Neutral")
                print(f"{i}. {title}")
                print(f"   → {link}")
                print(f"   Sentiment: {sentiment}\n")

    except Exception as e:
        print(f"❌ Error loading joblib file: {str(e)}")



📅 Last Updated: 2025-10-26T22:39:41.106689
📰 Source: CoinDesk
🧾 Total Articles: 50

1. Was the Democrat DeFi Doc Leaked in 'Bad Faith'? Senator Mark Warner on a Way Forward for Market Structure
   → No link found
   Sentiment: ⚪ Neutral

2. Featured Stories
   → No link found
   Sentiment: ⚪ Neutral

3. North Korea’s AI-Powered Hackers Are Redefining Crypto Crime
   → https://www.coindesk.com//business/2025/10/25/north-korea-s-ai-powered-hackers-are-redefining-crypto-crime
   Sentiment: ⚪ Neutral

4. Bitcoin Bid, XRP Retakes 200-Day Average as Fed Rate Cut Looms; 'Mag 7' Earnings, Trump-Xi Summit Eyed
   → https://www.coindesk.com//markets/2025/10/26/bitcoin-bid-xrp-tops-200-day-average-as-fed-rate-cut-looms-mag-7-earnings-trump-xi-summit-eyed
   Sentiment: ⚪ Neutral

5. North Korea’s AI-Powered Hackers Are Redefining Crypto Crime
   → https://www.coindesk.com//business/2025/10/25/north-korea-s-ai-powered-hackers-are-redefining-crypto-crime
   Sentiment: ⚪ Neutral

6. Bitmine’s Tom Lee